# Dog Breed Identification by Image Classification


The goal of this project is to be able to identify the breed of a dog given a picture of it. The data was provided by Kaggle for their dog breed identification contest. My approach is to use a convolutional neural network to classify these images. I will train a variety of networks of my own design and additionally apply some pre-trained networks and compare the results. 

In [2]:
#import relevant packages
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.applications import ResNet50
from keras import Model
from keras.preprocessing import image
import cv2
from sklearn.model_selection import train_test_split

In [4]:
#load the data
labels = pd.read_csv('../data/dogs/data/labels.csv.zip',compression='zip')
labels.head(2)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo


In [5]:
#The images are all different dimensions so this will be the parameter to
#resize them to be square 
image_size = 128

### Preparing the data to train the networks

In [6]:
def prepare_train_validation():
    #converting breed categories to categorical booleans
    one_hot_labels = pd.get_dummies(labels['breed']).values
    X_raw = []
    y_raw = []
    #loads and processes the images into image_size by image_size by 3 tensors
    for ind, row in enumerate(labels.values):
        img = cv2.imread('../data/dogs/data/train/{}.jpg'.format(row[0]))
        img = cv2.resize(img,(image_size,image_size))
        X_raw.append(img)
        y_raw.append(one_hot_labels[ind,:])
    X = np.array(X_raw)
    y =np.array(y_raw)
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2,stratify = y, random_state=1)
    return X_train, X_validation, y_train, y_validation

In [7]:
#creates training and validation sets with every class represented
#in both the training and validation sets.
X_train, X_validation, y_train, y_validation = prepare_train_validation()


In [6]:
#length of training data for use in determining batch size later
m_training = X_train.shape[0]

In [11]:
vgg_16_conv = VGG16(weights='imagenet',include_top = False)
img_in = Input(shape = (image_size,image_size,3),name = 'image_input')
outputVGG16 = vgg_16_conv(img_in)
X = Flatten()(outputVGG16)
X = Dense(2048, activation='relu',name="dense1")(X)
X = Dense(2048, activation='relu',name="dense2")(X)
X = Dense(120, activation='softmax', name="output")(X)
adapted_VGG16 = Model(input = img_in,output = X)
adapted_VGG16.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'] )


/home/connor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("ou...)`
  import sys


In [12]:
adapted_VGG16.fit(X_train,y_train,epochs = 10, batch_size = 19)

Epoch 1/10
 589/8177 [=>............................] - ETA: 1:46:07 - loss: 15.9163 - acc: 0.0119

KeyboardInterrupt: 

## First Experimental Network
this network is based on VGG16 but has been simplified 

In [ ]:
def conv_network_1( input_shape,weights_path = None):
    model = Sequential()
    
    model.add(ZeroPadding2D((1,1),input_shape=input_shape))
    
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    #model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(MaxPooling2D((2,2),strides = (2,2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu',name="dense1"))
    model.add(Dense(1024, activation='relu',name="dense2"))
    model.add(Dense(120, activation= 'softmax',name="dense3"))
    return model

In [ ]:
#Training the network using an rmsprop optimizer
model = conv_network_1(input_shape = (image_size,image_size,3))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'] )
model.fit(X_train,y_train,epochs = 10, batch_size = 19)

Epoch 1/10
 190/8177 [..............................] - ETA: 1:00:27 - loss: 15.1554 - acc: 0.0105 

In [ ]:
#for layer in model.layers:
#    print(layer.get_output_at(0).get_shape().as_list())
#model.fit(X,y,epochs = 10, batch_size = 38)

In [ ]:
def conv_network_2(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((1,1),input_shape=input_shape)(X_input)
    
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool1a')(X)
    
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool2a')(X)
    
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool3a')(X)
    X = Activation('relu')(X)
    
    X = Flatten()(X)
    X = Dense(120, activation='relu',name="dense1")(X)

    model = Model(inputs = X_input, outputs = X, name = 'convnet2')
    return model

In [ ]:
#model2 = conv_network_2(X.shape[1:])

In [ ]:
#model2.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',)

In [ ]:
#model2.fit(X,y,epochs = 10, batch_size = 38)